# SubSection Management Functions

## Setup

### Imports

In [11]:

from typing import Any
from typing import Dict, List, NamedTuple, TypeVar
from typing import Iterable, Any, Callable, Union, Generator
from typing import Tuple


import re
from pprint import pprint
from functools import partial
from abc import ABC, abstractmethod

from sections import Section
from sections import SectionBreak
from buffered_iterator import BufferedIterator
from sections import set_method, Rule, RuleSet
from sections import ProcessingMethods

# TestSingleLineStartSections

In [12]:
test_text = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'More text to be ignored',
    ]
start_sub_section = Section(
    name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before')
    )
test_iter = BufferedIterator(test_text)


Expect `['StartSection A']`

In [13]:
start_sub_section.read(test_iter)

['StartSection A']

Expect `['StartSection B']`

In [14]:
start_sub_section.read(test_iter)

['StartSection B']

Expect `[]`

In [15]:
start_sub_section.read(test_iter)

[]

# test_single_line_start_subsection

![Good](..\Valid.png) <br>
**Expected** `[['StartSection A'], ['StartSection B']]`<br>
**Got** `[['StartSection A'], ['StartSection B']]`


In [16]:
test_text = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'More text to be ignored',
    ]
start_sub_section = Section(
    name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before')
    )
full_section = Section(
    name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    processor=[start_sub_section]
    )
full_section.read(test_text)

[['StartSection A'], ['StartSection B']]

In [17]:
test_text = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'More text to be ignored',
    ]
start_sub_section = Section(
    name='SubSection',
    start_search=False,
    end_section=SectionBreak(True, break_offset='After')
    )
full_section = Section(
    name='Full',
    start_section=SectionBreak('ignored', break_offset='After'),
    end_section=SectionBreak('ignored', break_offset='Before'),
    processor=[start_sub_section]
    )
full_section.read(test_text)

[['StartSection A', 'EndSection A'], ['StartSection B', 'EndSection B']]

# test_two_single_line_subsections

In [18]:
test_text = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'More text to be ignored',
    ]
start_sub_section = Section(
    name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before')
    )
end_sub_section = Section(
    name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )
full_section = Section(
    name='Full',
    processor=[(start_sub_section, end_sub_section)]
    )
full_section.read(test_text)

[{'StartSubSection': ['StartSection A'], 'EndSubSection': ['EndSection A']},
 {'StartSubSection': ['StartSection B'], 'EndSubSection': ['EndSection B']}]

Expected
```
[{
    'StartSubSection': ['StartSection A'],
    'EndSubSection': ['EndSection A']
    },{
    'StartSubSection': ['StartSection B'], 
    'EndSubSection': ['EndSection B']
    }]
```

Expected Output:
```
[
    [['StartSection A', 'MiddleSection A', 'EndSection A']],
    [['StartSection B', 'MiddleSection B', 'EndSection B']],
    [['StartSection C', 'MiddleSection C', 'EndSection C']]
    ]
```

In [19]:
test_text = [
    'Text to be ignored',
    'StartSection A',
    'MiddleSection A',
    'EndSection A',
    'Unwanted text between sections',
    'StartSection B',
    'MiddleSection B',
    'EndSection B',
    'StartSection C',
    'MiddleSection C',
    'EndSection C',
    'Even more text to be ignored',
    ]
sub_section = Section(name='SubSection')
full_section = Section(
    name='Full', processor=sub_section,
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )
multi_section = Section(
    name='Multi',
    processor=full_section
    )
read_1 = multi_section.read(test_text)
read_1

[[['StartSection A', 'MiddleSection A', 'EndSection A']],
 [['StartSection B', 'MiddleSection B', 'EndSection B']],
 [['StartSection C', 'MiddleSection C', 'EndSection C']]]

### Test Text

In [20]:
test_text = [
            'Text to be ignored',
            'StartSection A',
            'MiddleSection A',
            'EndSection A',
            'Unwanted text between sections',
            'StartSection B',
            'MiddleSection B',
            'EndSection B',
            'StartSection C',
            'MiddleSection C',
            'EndSection C',
            'Even more text to be ignored',
            ]

short_multi_section_text = [
    'StartSection Name:A',
    'A Content1:a',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'EndSection Name:B'
    ]

multi_section_text = [
    'Text to be ignored',
    'StartSection Name:A',
    'A Content1:a',
    'B Content1:b',
    'C Content1:c',
    'EndSection Name:A',
    'StartSection Name:B',
    'A Content2:a',
    'B Content2:b',
    'C Content2:c',
    'EndSection Name:B',
    'Even more text to be ignored',
    ]


## Subsection Requirements
### Context
- Do not change the supplied context
- Use *self.context*
- *self.context* needs to be updated after every stage
- Need to protect standard context items from being changed by sub sections

### Source
- Supplied source needs to be isolated from the iterated source or it may exit too soon.
- Section's source pointer needs to be adjusted so that any "Future Items" are not missed.
- Both context and iterator should be isolated by default 


## Functions for subsections

In [21]:
def true_iterable(variable)-> bool:
    '''Indicate if the variable is a non-string type iterable.
    Arguments:
        variable {Iterable[Any]} -- The variable to test.
    Returns:
        True if variable is a non-string iterable.
    '''
    return not isinstance(variable, str) and isinstance(variable, Iterable)  # pylint: disable=isinstance-second-argument-not-valid-type



### Check for empty object

In [22]:
def is_empty(obj: Any)->bool:
    '''Test whether an object is empty.

    If object is None, it is empty.
    If object has length 0 it is empty.
    Otherwise it is not empty.

    Args:
        obj (Any): The object to be tested

    Returns:
        bool: Returns true if the object is empty.
    '''
    if obj is None:
        return True
    try:
        has_length = len(obj)
    except TypeError as err:
            return False
    else:
        if has_length == 0:
            return True
        else:
            return False

### Read a subsection

In [24]:
## Single line subsections
name_section = Section(
    name='Name',
    start_section=SectionBreak('StartSection', name='NameStart'),
    end_section=SectionBreak(True, name='NameEnd')
    )

content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', 
                             break_offset='Before', 
                             name='EndContent')
    )

end_section = Section(
    name='End',
    end_section=SectionBreak(True, name='EndEnd')
    )


### Check individual subsections

In [25]:
test_iter = BufferedIterator(multi_section_text)

print(name_section.read(test_iter),'\n')

print(content_section.read(test_iter),'\n')

print(end_section.read(test_iter),'\n')

print(repr(test_iter))

['StartSection Name:A'] 

['A Content1:a', 'B Content1:b', 'C Content1:c'] 

['EndSection Name:A'] 

BufferedIterator(source=<list_iterator object at 0x0000023A6B14EB50>, buffer_size=5)
	BufferedIterator.previous_items = deque(['A Content1:a', 'B Content1:b', 'C Content1:c', 'EndSection Name:A'], maxlen=5)
	BufferedIterator.future_items = deque(['StartSection Name:B'], maxlen=5)
	BufferedIterator.item_count = 6
	BufferedIterator.status = ACTIVE


### Single Line Subsections as a group

In [26]:
section_list = (name_section, content_section, end_section)

In [27]:
test_iter = BufferedIterator(multi_section_text)
context = {'dummy': 'asdf'}

combined_sections = [grp for grp in assemble_subsection_group(section_list,
                                                              test_iter,
                                                              context)]

pprint(combined_sections)
print()
pprint(context)

NameError: name 'assemble_subsection_group' is not defined

# Done to Here

Generator exits are captured by the read method.
`source.status` provides an indication of whether the
iterator has been exhausted.
```
if source.status in 'Completed':
    done=True
```  

However if `source` is not a BufferedIterator then it may not contain a 
`source.status` attribute.  


Try `inspect.gi_running`

In [ ]:
import inspect

In [28]:
multi_section_text

['Text to be ignored',
 'StartSection Name:A',
 'A Content1:a',
 'B Content1:b',
 'C Content1:c',
 'EndSection Name:A',
 'StartSection Name:B',
 'A Content2:a',
 'B Content2:b',
 'C Content2:c',
 'EndSection Name:B',
 'Even more text to be ignored']

![Error](..\Error.png) There are 4 empty entries. only one is expected

### Generator function to step through entire source returning section groups

In [29]:
test_iter = BufferedIterator(multi_section_text)
context = {'dummy': 'asdf'}

s_iter = assemble_subsection_group(section_list, test_iter, context)
print(inspect.getgeneratorstate(s_iter))
combined_sections = []
while True:
    try:
        combined_sections.append(next(s_iter))
        print(inspect.getgeneratorstate(s_iter))
    except StopIteration:
        print(inspect.getgeneratorstate(s_iter))
        break
   

pprint(combined_sections)


NameError: name 'assemble_subsection_group' is not defined

In [ ]:
test_iter = BufferedIterator(multi_section_text)
context = {'dummy': 'asdf'}

s_iter = assemble_subsection_group(section_list, test_iter, context)
print(inspect.getgeneratorstate(s_iter))
combined_sections = []
while True:
    try:
        itm = next(s_iter)
        if not is_empty(itm):
            # Don't return empty section group.
            combined_sections.append(itm)
        print('\n\nOutput:')
        pprint(itm)
        print('\ncontext:')
        pprint(context,indent=4)
        print('\nSource:')
        print(repr(test_iter))
    except StopIteration:
        print(inspect.getgeneratorstate(s_iter))
        break
   

pprint(combined_sections)


### Sections with the same name

In [ ]:
## Single line subsections
name_section = Section(
    name='subsection',
    start_section=SectionBreak('StartSection', name='NameStart'),
    end_section=SectionBreak(True, name='NameEnd')
    )

content_section = Section(
    name='subsection',
    end_section=SectionBreak('EndSection', 
                             break_offset='Before', 
                             name='EndContent')
    )

end_section = Section(
    name='subsection2',
    end_section=SectionBreak(True, name='EndEnd')
    )

subsection_group = (name_section, content_section, end_section)

In [ ]:
from collections import Counter

In [ ]:
section_names = Counter([sub_rdr.name for sub_rdr in subsection_group])
section_names

In [ ]:
len(section_names) < len(subsection_group)

In [ ]:
name_count = {name: 0 for name in section_names.keys()}
name_count

In [ ]:
a = iter(subsection_group)

In [ ]:
sub_sec = next(a)
name = sub_sec.name
name

In [ ]:
section_names[name]

In [ ]:
unique_names = len(section_names)
unique_names

## Use this to test subsections with a non 1:1 processor

In [ ]:
'''Testing Source and Section item counting.

Process method should track the number of Source lines used for each processed line

Processor creates sequence of source.item_count for each output item
- Len(section.item_count) = # processed items
- section.item_count[-1] = # source items (includes skipped source items)
- Property item_count returns len(self._item_count)
- Property source_item_count returns self._item_count[-1]
'''

# %% Imports
import unittest
from pprint import pprint
import random
from buffered_iterator import BufferedIterator

from sections import SectionBreak, Section
from sections import Rule, RuleSet, ProcessingMethods
# %% Logging
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
logger = logging.getLogger('Source Tracking Tests')
logger.setLevel(logging.DEBUG)
#logger.setLevel(logging.INFO)

# %% Processing Functions
def pairs(source):
    '''Convert a sequence of items into a sequence of item pairs

    Successive items are combined into length 2 tuples.

    Args:
        source (Sequence): any sequence of hashable items

    Yields:
        Tuple[Any]: Successive items combined into length 2 tuples.
    '''
    for item in source:
        yield tuple([item, next(source)])


def n_split(source):
    '''Extract numbers from stings of comma separated integers.

    Number are extracted by splitting on the commas.  Spaces are ignored.

    Args:
        source (Sequence[str]): A sequence of stings composed of comma separated
            integers. e.g. ['0, 1', '2, 3', '4, 5' ...]

    Yields:
        int: Integer values extracted from the strings.
    '''
    for item in source:
        nums = [int(num_s.strip()) for num_s in item.split(',')]
        yield from nums


def odd_nums(source):
    '''Yield Odd items
    Args:
        source (Sequence[int]): A sequence of integers

    Yields:
        int: odd integers from the source
    '''
    for item in source:
        if int(item)%2 == 1:
            yield item

# %% Test Source Tracking
class TestSourceTracking(unittest.TestCase):
    def setUp(self):
        self.buffer_size = 5
        self.num_items = 10

        self.str_source = BufferedIterator(
            (str(i) for i in range(self.num_items)),
            buffer_size=self.buffer_size)

        self.int_source = BufferedIterator(
            (i for i in range(self.num_items)),
            buffer_size=self.buffer_size)

        self.pairs_source = BufferedIterator(
            [f'{a}, {b}' for a, b in zip(range(0, self.num_items * 2, 2),
                                         range(1, self.num_items * 2, 2))],
            buffer_size=self.buffer_size)

    def test_before_source_initialized(self):
        '''Before source initialized
            - Section.source_index is None
            - Section.source_item_count is 0
            - Section.item_count is 0
        '''
        empty_section = Section(name='empty')
        source_index = empty_section.source_index
        source_item_count = empty_section.source_item_count
        item_count = empty_section.item_count
        self.assertIsNone(source_index)
        self.assertEqual(source_item_count, 0)
        self.assertEqual(item_count, 0)

    def test_source_beginning(self):
        '''At beginning of source
            - Section.source_index is empty list
            - Section.source_item_count is 0
            - Section.item_count is 0
        '''
        not_started_section = Section(name='Not Started')
        not_started_section.source = self.int_source
        source_index = not_started_section.source_index
        source_item_count = not_started_section.source_item_count
        item_count = not_started_section.item_count
        self.assertEqual(source_index, [0])
        self.assertEqual(source_item_count, 0)
        self.assertEqual(item_count, 0)

    def test_1_to_1_processor(self):
        '''1-to-1 match
            - range(n) as source
            - processor just returns item
            - for each section item:
            - source.item_count = item = section.source_item_count
            - source.item_count = section.item_count
        '''
        section_1_1 = Section(name='1-to-1 match')
        for item in section_1_1.process(self.int_source):
            with self.subTest(item=item):
                source_count = self.int_source.item_count
                source_item_count = section_1_1.source_item_count
                item_count = section_1_1.item_count
                self.assertEqual(item+1, source_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_count, item_count)

    def test_2_to_1_processor(self):
        '''2-to-1 match
            - `range(n)` as source
            - processor converts 2 successive source items into tuple of
            length 2.
            - for each section item:
                - item = (source.item_count-2, source.item_count-1)
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count * 2
        '''
        section_2_1 = Section(name='2-to-1 match',
                              processor=[pairs])
        for item in section_2_1.process(self.int_source):
            with self.subTest(item=item):
                source_count = self.int_source.item_count
                source_item_count = section_2_1.source_item_count
                item_count = section_2_1.item_count
                expected_item = (source_count-2, source_count-1)
                self.assertEqual(item, expected_item)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_item_count, item_count * 2)

    def test_1_to_2_processor(self):
        '''1-to-2 match
        - Numerical pairs as source:
            `['0, 1', '2, 3', '4, 5'` $\cdots$`]`
        - processor converts 1 source item into 2 output lines
        - for each source item:
            > `nums = [int(num_s.strip()) for num_s in item.split(',')]`<br>
            > `section item 1 = nums[0]`,<br>
            > `section item 2 = nums[1]`
        - source.item_count = (section.item_count + 1) // 2
        - section.source_item_count = section.source_index[-1]
        '''
        section_1_2 = Section(name='1-to-2 match',
                              processor=[n_split])
        for item in section_1_2.process(self.pairs_source):
            with self.subTest(item=item):
                source_count = self.pairs_source.item_count
                source_item_count = section_1_2.source_item_count
                item_count = section_1_2.item_count
                self.assertEqual(item+1, item_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_count, (item_count+1) //2)

    def test_skip_first_item_count(self):
        '''Skip First Source Item
            - (str(i) for i in range(n)) as source
            - start_section='1', offset='Before'
            - processor returns int(item)
            - for each section item:
                - source.item_count = item
                - source.item_count = section.source.item_count
                - source.item_count = section.item_count
        '''
        section_skip_0 = Section(
            name='Skipped First Source Item',
            start_section=SectionBreak('1', break_offset='Before')
            )
        for item in section_skip_0.process(self.str_source):
            with self.subTest(item=item):
                source_count = self.str_source.item_count
                source_item_count = section_skip_0.source_item_count
                item_count = section_skip_0.item_count
                self.assertEqual(int(item)+1, source_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(int(item), item_count)

    def test_skip_two_item_counts(self):
        '''Skip First 2 Source Items
            - (str(i) for i in range(n)) as source
            - start_section='1', offset='After'
            - processor returns int(item)
            - for each section item:
                - source.item_count = item + 1
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count + 2
        '''
        section_skip_2 = Section(
            name='Skipped First Source Item',
            start_section=SectionBreak('1', break_offset='After')
            )
        for item in section_skip_2.process(self.str_source):
            with self.subTest(item=item):
                source_count = self.str_source.item_count
                source_item_count = section_skip_2.source_item_count
                item_count = section_skip_2.item_count
                self.assertEqual(int(item)+1, source_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_count, item_count + 2)

    def test_do_not_count_dropped_items(self):
        '''Don't Count Dropped Items
            - range(n) as source
            - processor drops even items and yields odd items
            - for each section item:
                - item + 1 = source.item_count
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count * 2
        '''
        section_odd = Section(
            name='Odd Numbers',
            processor=[odd_nums]
            )
        for item in section_odd.process(self.int_source):
            with self.subTest(item=item):
                source_count = self.int_source.item_count
                source_item_count = section_odd.source_item_count
                item_count = section_odd.item_count
                self.assertEqual(item+1, source_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_count, item_count * 2)

    def test_completed_section_item_count(self):
        '''Completed Section Item Count
            - (str(i) for i in range(n)) as source
            - processor drops even items and yields odd items
            - after section.read(source):
                - source.item_count = section.source_item_count
                - section.source_item_count = section.item_count = n * 2
        '''
        section_odd = Section(
            name='Odd Numbers',
            processor=[odd_nums]
            )
        section_odd.read(self.int_source)
        source_count = self.int_source.item_count
        source_item_count = section_odd.source_item_count
        item_count = section_odd.item_count
        self.assertEqual(source_count, source_item_count)
        self.assertEqual(source_count, item_count * 2)

    def test_completed_section_partial_source_item_count(self):
        '''Partial Source Completed Section
            - (str(i) for i in range(n)) as source
            - Random start_section and end_section
            - after section.read(source):
                - source.item_count = section.source_item_count
                - source.item_count = end_num
                - section.item_count = end_num - start_num
        '''
        start_num = random.randint(1, self.num_items-2)
        end_num = random.randint(start_num + 1, self.num_items)
        part_section = Section(
            name='Partial Source Section',
            start_section=str(start_num),
            end_section=str(end_num)
            )
        part_section.read(self.str_source)
        source_count = self.str_source.item_count
        source_item_count = part_section.source_item_count
        item_count = part_section.item_count
        self.assertEqual(source_count, source_item_count)
        self.assertEqual(source_count, end_num)
        self.assertEqual(item_count, end_num-start_num)

    def test_completed_section_partial_source_with_end_before_item_count(self):
        '''Completed Section With End Before
            - `(str(i) for i in range(n))` as source
            - end_section='2', offset='Before'
            - after section.read(source):
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count = 2
        '''
        section_end_before = Section(
            name='End Before',
            end_section=SectionBreak('2', break_offset='Before')
            )

        item_list = section_end_before.read(self.str_source)
        source_count = self.str_source.item_count
        source_item_count = section_end_before.source_item_count
        item_count = section_end_before.item_count

        self.assertEqual(source_count, 2)
        self.assertEqual(source_item_count, 2)
        self.assertEqual(item_count, 2)
        self.assertEqual(item_list, ['0', '1'])

    def test_completed_section_partial_source_with_end_after_item_count(self):
        '''Completed Section With End After
            - `(str(i) for i in range(n))` as source
            - end_section='2', offset='After'
            - after section.read(source):
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count = 3
        '''
        section_end_before = Section(
            name='End Before',
            end_section=SectionBreak('2', break_offset='After')
            )

        item_list = section_end_before.read(self.str_source)
        source_count = self.str_source.item_count
        source_item_count = section_end_before.source_item_count
        item_count = section_end_before.item_count

        self.assertEqual(source_count, 3)
        self.assertEqual(source_item_count, 3)
        self.assertEqual(item_count, 3)
        self.assertEqual(item_list, ['0', '1', '2'])


if __name__ == '__main__':
    unittest.main()


## Use this to test subsections with a non 1:1 processor

In [ ]:
'''Testing Source and Section item counting.

Process method should track the number of Source lines used for each processed line

Processor creates sequence of source.item_count for each output item
- Len(section.item_count) = # processed items
- section.item_count[-1] = # source items (includes skipped source items)
- Property item_count returns len(self._item_count)
- Property source_item_count returns self._item_count[-1]
'''

# %% Imports
import unittest
from pprint import pprint
import random
from buffered_iterator import BufferedIterator

from sections import SectionBreak, Section
from sections import Rule, RuleSet, ProcessingMethods
# %% Logging
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
logger = logging.getLogger('Source Tracking Tests')
logger.setLevel(logging.DEBUG)
#logger.setLevel(logging.INFO)

# %% Processing Functions
def pairs(source):
    '''Convert a sequence of items into a sequence of item pairs

    Successive items are combined into length 2 tuples.

    Args:
        source (Sequence): any sequence of hashable items

    Yields:
        Tuple[Any]: Successive items combined into length 2 tuples.
    '''
    for item in source:
        yield tuple([item, next(source)])


def n_split(source):
    '''Extract numbers from stings of comma separated integers.

    Number are extracted by splitting on the commas.  Spaces are ignored.

    Args:
        source (Sequence[str]): A sequence of stings composed of comma separated
            integers. e.g. ['0, 1', '2, 3', '4, 5' ...]

    Yields:
        int: Integer values extracted from the strings.
    '''
    for item in source:
        nums = [int(num_s.strip()) for num_s in item.split(',')]
        yield from nums


def odd_nums(source):
    '''Yield Odd items
    Args:
        source (Sequence[int]): A sequence of integers

    Yields:
        int: odd integers from the source
    '''
    for item in source:
        if int(item)%2 == 1:
            yield item

# %% Test Source Tracking
class TestSourceTracking(unittest.TestCase):
    def setUp(self):
        self.buffer_size = 5
        self.num_items = 10

        self.str_source = BufferedIterator(
            (str(i) for i in range(self.num_items)),
            buffer_size=self.buffer_size)

        self.int_source = BufferedIterator(
            (i for i in range(self.num_items)),
            buffer_size=self.buffer_size)

        self.pairs_source = BufferedIterator(
            [f'{a}, {b}' for a, b in zip(range(0, self.num_items * 2, 2),
                                         range(1, self.num_items * 2, 2))],
            buffer_size=self.buffer_size)

    def test_before_source_initialized(self):
        '''Before source initialized
            - Section.source_index is None
            - Section.source_item_count is 0
            - Section.item_count is 0
        '''
        empty_section = Section(name='empty')
        source_index = empty_section.source_index
        source_item_count = empty_section.source_item_count
        item_count = empty_section.item_count
        self.assertIsNone(source_index)
        self.assertEqual(source_item_count, 0)
        self.assertEqual(item_count, 0)

    def test_source_beginning(self):
        '''At beginning of source
            - Section.source_index is empty list
            - Section.source_item_count is 0
            - Section.item_count is 0
        '''
        not_started_section = Section(name='Not Started')
        not_started_section.source = self.int_source
        source_index = not_started_section.source_index
        source_item_count = not_started_section.source_item_count
        item_count = not_started_section.item_count
        self.assertEqual(source_index, [0])
        self.assertEqual(source_item_count, 0)
        self.assertEqual(item_count, 0)

    def test_1_to_1_processor(self):
        '''1-to-1 match
            - range(n) as source
            - processor just returns item
            - for each section item:
            - source.item_count = item = section.source_item_count
            - source.item_count = section.item_count
        '''
        section_1_1 = Section(name='1-to-1 match')
        for item in section_1_1.process(self.int_source):
            with self.subTest(item=item):
                source_count = self.int_source.item_count
                source_item_count = section_1_1.source_item_count
                item_count = section_1_1.item_count
                self.assertEqual(item+1, source_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_count, item_count)

    def test_2_to_1_processor(self):
        '''2-to-1 match
            - `range(n)` as source
            - processor converts 2 successive source items into tuple of
            length 2.
            - for each section item:
                - item = (source.item_count-2, source.item_count-1)
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count * 2
        '''
        section_2_1 = Section(name='2-to-1 match',
                              processor=[pairs])
        for item in section_2_1.process(self.int_source):
            with self.subTest(item=item):
                source_count = self.int_source.item_count
                source_item_count = section_2_1.source_item_count
                item_count = section_2_1.item_count
                expected_item = (source_count-2, source_count-1)
                self.assertEqual(item, expected_item)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_item_count, item_count * 2)

    def test_1_to_2_processor(self):
        '''1-to-2 match
        - Numerical pairs as source:
            `['0, 1', '2, 3', '4, 5'` $\cdots$`]`
        - processor converts 1 source item into 2 output lines
        - for each source item:
            > `nums = [int(num_s.strip()) for num_s in item.split(',')]`<br>
            > `section item 1 = nums[0]`,<br>
            > `section item 2 = nums[1]`
        - source.item_count = (section.item_count + 1) // 2
        - section.source_item_count = section.source_index[-1]
        '''
        section_1_2 = Section(name='1-to-2 match',
                              processor=[n_split])
        for item in section_1_2.process(self.pairs_source):
            with self.subTest(item=item):
                source_count = self.pairs_source.item_count
                source_item_count = section_1_2.source_item_count
                item_count = section_1_2.item_count
                self.assertEqual(item+1, item_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_count, (item_count+1) //2)

    def test_skip_first_item_count(self):
        '''Skip First Source Item
            - (str(i) for i in range(n)) as source
            - start_section='1', offset='Before'
            - processor returns int(item)
            - for each section item:
                - source.item_count = item
                - source.item_count = section.source.item_count
                - source.item_count = section.item_count
        '''
        section_skip_0 = Section(
            name='Skipped First Source Item',
            start_section=SectionBreak('1', break_offset='Before')
            )
        for item in section_skip_0.process(self.str_source):
            with self.subTest(item=item):
                source_count = self.str_source.item_count
                source_item_count = section_skip_0.source_item_count
                item_count = section_skip_0.item_count
                self.assertEqual(int(item)+1, source_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(int(item), item_count)

    def test_skip_two_item_counts(self):
        '''Skip First 2 Source Items
            - (str(i) for i in range(n)) as source
            - start_section='1', offset='After'
            - processor returns int(item)
            - for each section item:
                - source.item_count = item + 1
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count + 2
        '''
        section_skip_2 = Section(
            name='Skipped First Source Item',
            start_section=SectionBreak('1', break_offset='After')
            )
        for item in section_skip_2.process(self.str_source):
            with self.subTest(item=item):
                source_count = self.str_source.item_count
                source_item_count = section_skip_2.source_item_count
                item_count = section_skip_2.item_count
                self.assertEqual(int(item)+1, source_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_count, item_count + 2)

    def test_do_not_count_dropped_items(self):
        '''Don't Count Dropped Items
            - range(n) as source
            - processor drops even items and yields odd items
            - for each section item:
                - item + 1 = source.item_count
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count * 2
        '''
        section_odd = Section(
            name='Odd Numbers',
            processor=[odd_nums]
            )
        for item in section_odd.process(self.int_source):
            with self.subTest(item=item):
                source_count = self.int_source.item_count
                source_item_count = section_odd.source_item_count
                item_count = section_odd.item_count
                self.assertEqual(item+1, source_count)
                self.assertEqual(source_count, source_item_count)
                self.assertEqual(source_count, item_count * 2)

    def test_completed_section_item_count(self):
        '''Completed Section Item Count
            - (str(i) for i in range(n)) as source
            - processor drops even items and yields odd items
            - after section.read(source):
                - source.item_count = section.source_item_count
                - section.source_item_count = section.item_count = n * 2
        '''
        section_odd = Section(
            name='Odd Numbers',
            processor=[odd_nums]
            )
        section_odd.read(self.int_source)
        source_count = self.int_source.item_count
        source_item_count = section_odd.source_item_count
        item_count = section_odd.item_count
        self.assertEqual(source_count, source_item_count)
        self.assertEqual(source_count, item_count * 2)

    def test_completed_section_partial_source_item_count(self):
        '''Partial Source Completed Section
            - (str(i) for i in range(n)) as source
            - Random start_section and end_section
            - after section.read(source):
                - source.item_count = section.source_item_count
                - source.item_count = end_num
                - section.item_count = end_num - start_num
        '''
        start_num = random.randint(1, self.num_items-2)
        end_num = random.randint(start_num + 1, self.num_items)
        part_section = Section(
            name='Partial Source Section',
            start_section=str(start_num),
            end_section=str(end_num)
            )
        part_section.read(self.str_source)
        source_count = self.str_source.item_count
        source_item_count = part_section.source_item_count
        item_count = part_section.item_count
        self.assertEqual(source_count, source_item_count)
        self.assertEqual(source_count, end_num)
        self.assertEqual(item_count, end_num-start_num)

    def test_completed_section_partial_source_with_end_before_item_count(self):
        '''Completed Section With End Before
            - `(str(i) for i in range(n))` as source
            - end_section='2', offset='Before'
            - after section.read(source):
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count = 2
        '''
        section_end_before = Section(
            name='End Before',
            end_section=SectionBreak('2', break_offset='Before')
            )

        item_list = section_end_before.read(self.str_source)
        source_count = self.str_source.item_count
        source_item_count = section_end_before.source_item_count
        item_count = section_end_before.item_count

        self.assertEqual(source_count, 2)
        self.assertEqual(source_item_count, 2)
        self.assertEqual(item_count, 2)
        self.assertEqual(item_list, ['0', '1'])

    def test_completed_section_partial_source_with_end_after_item_count(self):
        '''Completed Section With End After
            - `(str(i) for i in range(n))` as source
            - end_section='2', offset='After'
            - after section.read(source):
                - source.item_count = section.source_item_count
                - source.item_count = section.item_count = 3
        '''
        section_end_before = Section(
            name='End Before',
            end_section=SectionBreak('2', break_offset='After')
            )

        item_list = section_end_before.read(self.str_source)
        source_count = self.str_source.item_count
        source_item_count = section_end_before.source_item_count
        item_count = section_end_before.item_count

        self.assertEqual(source_count, 3)
        self.assertEqual(source_item_count, 3)
        self.assertEqual(item_count, 3)
        self.assertEqual(item_list, ['0', '1', '2'])


if __name__ == '__main__':
    unittest.main()


## Single line subsections

In [ ]:
name_section = Section(
    name='Name',
    start_section=SectionBreak('StartSection', name='NameStart'),
    end_section=SectionBreak(True, name='NameEnd')
    )

content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', 
                             break_offset='Before', 
                             name='EndContent')
    )

end_section = Section(
    name='End',
    end_section=SectionBreak(True, name='EndEnd')
    )


In [ ]:
section_list = (name_section, content_section, end_section)

subsection_group = ProcessingMethods(section_list)

full_section = Section(
    name='Full Section',
    processor=subsection_group
    )



In [ ]:
test_iter = BufferedIterator(multi_section_text)
context = {'dummy': 'asdf'}

full_section.read(test_iter)


In [ ]:

pprint(combined_sections)
print()
pprint(context)

#### Subsection definitions

In [ ]:
name_section = Section(
    name='Name',
    start_section=SectionBreak('Name', name='Name'),
    end_section=SectionBreak(True, name='EndName')
    )

content_section = Section(
    name='Content',
    end_section=SectionBreak('EndSection', 
                             break_offset='Before', 
                             name='EndContent')
    )

end_section = Section(
    name='End',
    end_section=SectionBreak(True, name='EndEnd')
    )

section_list = [name_section, content_section, end_section]


In [ ]:
from functools import partial
proc_func = partial(read_section_list, section_list)
test_context = {'dummy': 'Test'}

full_section = Section(
    name='Full Subsection',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    processor=proc_func
    )

full_section.read(test_iter, start_search=False)

In [ ]:
read_section_list(section_list, test_iter)

In [ ]:

print(repr(test_iter))